# Focal Loss

## Class Imbalance

간단하게 설명해서, 이미지 하나당 40000~50000개 정도를 추천하는데, 이중에 object가 있을거라 추정하는 위치는 매우 적습니다. <br>
문제는 다음과 같습니다.

사진속 대부분은 객체가 아닌 배경입니다. 문제는 실제 객체보다 배경이 압도적으로 더 많다는 것입니다. <br>
그 중에서 easy negatives(즉 background라고 쉽게 판단되는 부분들)은 매우 드물게 나오는 실제 객체들의 loss값을 압도해버리기 때문에 학습을 하는데 도움을 주지 못합니다. (그냥 학습이 잘 안됩니다.)

기계학습에서는 이렇게 class imbalance 문제가 일어나게 되면 2가지 방식으로 해결을 할 수 있습니다. <br>
첫번째는 downsampling 이고, 두번째는 oversampling 입니다. <br>
Downsampling은 압도적으로 많은 class를 적은 class의 비율에 맞춰서 줄여버리는 것이고, Oversampling은 적은 클래스를 압도적으로 많은 클래에 맞도록 양을 늘려버리는 것입니다. 이 2가지 방법중에 Focal Loss는 전자인 downsampling방법을 선택하였습니다. 

Focal Loss를 사용하면 잘 분류된 것(well-classified cases -> 예측값이 대충 0.5 이상인것들..)은 loss값이 점점 더 작아지고, 잘 분류하지 못한것들에는 loss값이 더 커지게 됩니다. 이는 분류하기 쉬운 배경화면들을 가중치를 덜 주고, rare하게 나오는 객체를 제대로 못맞추면 더 크게 가중치를 줘서 loss값을 높이려는 전략입니다.

## Cross Entropy

Focal Loss은 one-stage object detection 시나리오를 다루기 위해서 디자인 되었으며 학습도중의 foreground 그리고 background classes의 극단적인 class imbalance 문제를 해결합니다.

먼저 binary cross entropy를 시작으로 해서 focal loss를 정의하도록 합니다.

$$ \text{CE}(p, y) = 
\begin{cases} 
- \log(p) & \text{if}\ y = 1 \\ 
- \log(1-p) & \text{otherwise} \\
\end{cases} $$

* $ y \in \{ \pm 1 \} $  :  y 는  1 또는 -1 이며, ground-truth class를 나타낸다.
* $ p \in [ 0 , 1 ]  $ : p는 0 ~ 1 사이의 값이며, 모델이 추정한 확률값 (estimated probability) 입니다 (1에 가까울수록 ground-truth class)

notation을 줄여서 쓰기 위해서 $ P_t $ 를 다음과 같이 정의 합니다.

$$ P_t = 
\begin{cases} 
p & \text{if } y = 1 \\
1 - p & \text{otherwise}
\end{cases} $$

$$ CE(p, y) = CE(p_t) = -log(p_t) $$

## Balanced Cross Entropy

class imbalance문제를 해결하는 가장 일반적인 방법은 weighting factor $ \alpha \in [0, 1] $ 를 class 1 (객체)에 그리고 $ 1 - \alpha $ 를 class -1 (배경)에 적용하는 것 입니다.<br>
페이퍼에서는 $ \alpha = 0.25 $ 를 사용하였습니다.


$$ \alpha_t = 
\begin{cases} 
\alpha & \text{if } y = 1 \\
1 - \alpha & \text{otherwise}
\end{cases} $$

$$ \text{CE}(p_t) = - \alpha_t \log(p_t) $$

## Focal Loss Definition

페이퍼의 연구 결과가 말해주듯이, 대부분의 loss값 그리고 대부분의 gradient값을 차지하는 것은 easily classified negatives입니다. <br>
$ \alpha $ 를 사용해서 positive/negative examples 의 중요도를 조정할 수 있지만, easy/hard examples은 구별하지 못합니다. 

Focal loss에서는 easy examples은 down-weight를 주고, hard examples에 집중할 수 있도록 해줍니다. <br>
구체적으로는 modulating factor $ (1 - p_t)^\gamma $ 를 cross entropy에 추가를 합니다. 이때 $ \gamma \ge 0 $ 입니다. 

$$ \text{FL}(p_t) = - (1-p_t)^\gamma \log(p_t) $$

아래는 $ \gamma \in [0, 5] $ 값에 따른 그래프의 변화입니다.

![Focal Loss](images/retina-focal-loss-graph.png)

<span style="color:red">
**쉽게 설명하면!** 예측한 값이 ground-truth class에 맞을수록 영향도를 적게 주고, <br>
object를 background라고 하거나 background 를 object라고 한것처럼 개틀린것은 포커스하겠다는 뜻입니다.</span>

아래 코드를 보면.. P값이 높을수록 ground-truth class에 맞는건데, factor값이 0에 가깝습니다.<br>
예를 들어서 P값이 0.99 는 factor 값이 0.000001이고, P값이 0.01은 factor값이 0.97 입니다.

> **논문에서는 $ \gamma = 2 $ 일때 가장 학습이 잘 되었다고 합니다.**

In [107]:
import numpy as np
import pandas as pd

def focal_loss(y_true, y_pred, gamma=1, visual=True):
    p = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
    factor = (1 - p)**gamma
    loss = - (factor * np.log(factor))
    
    # Visualize    
    if visual:
        df = pd.DataFrame({'y_true': y_true,
                           'y_pred': y_pred,
                           'p': p,
                           'factor': factor,
                           'loss': loss}, columns=['y_true', 'y_pred', 'p', 'factor', 'loss'])
        display(df)
    return loss.mean()

y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.float64)
y_pred = np.array([0.99, 0.6, 0.3, 0.01, 0.01, 0.3, 0.6, 0.99], dtype=np.float64)
print('loss:', focal_loss(y_true, y_pred, 3))

,y_true,y_pred,p,factor,loss
0,0.0,0.99,0.01,0.970299,0.029255
1,0.0,0.60,0.40,0.216000,0.331015
2,0.0,0.30,0.70,0.027000,0.097522
3,0.0,0.01,0.99,0.000001,0.000014
4,1.0,0.01,0.01,0.970299,0.029255
5,1.0,0.30,0.30,0.343000,0.367019
6,1.0,0.60,0.60,0.064000,0.175928
7,1.0,0.99,0.99,0.000001,0.000014


loss: 0.12875271939682145


## Focal Loss with Alpha-balanced variant

<span style="color:red">**결론적으로 아래의 loss function을 실제 구현시에 사용합니다. **</span><br>
실제 focal loss를 적용할때는 $ \alpha $-balanced variant 를 함께 넣습니다. 

$$ \text{FL}(p_t) = - \alpha_t (1-p_t)^\gamma \log(p_t) $$

또한 실제 구현시에는 $ p $ 값에 sigmoid operation을 적용해줍니다. 

# Focal Loss Examples

## Easy Correctly Classified Example

예를 들어서 easily classified foreground object가 $ p = 0.9 $  를 갖고 있을때 cross entropy는 다음과 같습니다.

$$ \text{CE(foreground)} = -log(0.9) = 0.10536 $$

Easily background object 가 $ p = 0.1 $ 을 갖고 있을때 일발적인 cross entropy는 다음과 같습니다. 

$$ \text{CE(background)} = -log(1 - 0.1) = 0.10536 $$

두개의 값을 보면 서로 같은 값을 갖고 있습니다. 

Focal Loss를 적용해보도록 하겠습니다. (이때 alpha = 0.25 그리고 gamma = 2 적용)

$$ \begin{align} 
FL(foreground) &= -1 * 0.25 * (1 - 0.9)^2 \cdot \log(0.9) \\
&= -1 * 0.25 * 0.1^2 \cdot \log(0.9) \\
&=  0.00026
\end{align} $$

$$ \begin{align} 
FL(background) &= -1 * (1-0.25) * (1 - (1 - 0.1))^2 \cdot \log(1 - 0.1) \\
&= -1 * 0.75 * 0.1^2 \cdot \log(0.9) \\
&=  0.00079
\end{align} $$

결론적으로 다음과 같이 줄었습니다. 

$$ \begin{align} 
g(foreground) = \frac{0.10536}{0.00026} = 405 \\
g(background) = \frac{0.10536}{0.00079} = 133
\end{align} $$

즉 기존 cross entropy보다 foreground를 틀리면 405배 줄어들고, background를 틀리면 133배 줄어듭니다.

## Misclassified Example

예를 들어서 객체를 $ p = 0.1 $ 값으로 배경으로 잘못 보았다고 가정합니다. <br>
이때 일반적인 cross entropy는 다음과 같습니다. 

$$ \text{CE(foreground)} = -\log(0.1) = 2.3025 $$

$$ \text{CE(background)} = -\log(1 - 0.9) = 2.3025 $$

반면 focal loss의 경우는 다음과 같습니다. 

$$ \begin{align} 
FL(foreground) &= -1 * 0.25 * (1 - 0.1)^2 \cdot \log(0.1) \\
&= -1 * 0.25 * 0.9^2 \cdot \log(0.1) \\
&=  0.4667
\end{align} $$

$$ \begin{align} 
FL(background) &= -1 * (1-0.25) * (1 - (1 - 0.9))^2 \cdot \log(1 - 0.9) \\
&= -1 * 0.75 * 0.9^2 \cdot \log(0.1) \\
&=  1.3988
\end{align} $$

결론적으로 다음과 같이 줄었습니다 .

$$ \begin{align} 
g(foreground) = \frac{2.3025}{0.4667} = 4.9 \\
g(background) = \frac{2.3025}{1.3988} = 1.6460
\end{align} $$

즉 기존 cross entropy보다 foreground를 틀리면 4.9배 줄어들고, background를 틀리면 1.6배 줄어듭니다.

## Conclusion

| Senario | Type | Comparison | 
|:--------|:-----|:-----|
| Senario 1 (well classified) | Foreground | 405 |
|           | Background | 133 |
| Senario 2 (bad classified) | Foreground | 4.9 |
|           | Background | 1.6 |

결론적으로 기존의 cross entropy와 비교하였을때 매우 잘 classified된 곳에는 405배 또는 133배처럼 loss값이 줄어들었습니다. <br>
그리고 잘못 분류를 하였을때는 4.9배 또는 1.6배처럼 잘 분류했을때보다 덜 줄어들었습니다. 

즉!! Focal loss는 잘 분류가 되는 케이스에 관해서는 loss값이 매우 낮아지고, 잘못 분류를 한 케이스에는 loss값을 크게 잡아줍니다. 

# Feature Pyramid Networks

* [Understanding Feature Pyramid Networks for object detection (FPN)](https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c) 에서 내용을 참고 하였습니다. 


Feature pyramids는 서로 다른 크기의 객체를 인식하는데 사용되는 방법입니다. 다만 잘 사용되지 않았는데.. computation그리고 메모리가 너무 많이 사용되기 때문이기 때문입니다. 

![Feature Pyramids](images/feature-pyramids.png)

* **(a) Featurized image pyramid:** 과거에 사용한 방법으로, 이미지 자체를 피라미드화해서 각각 연산을 합니다. 물론 예측률이 높지만 연산비용이 매우 높습니다. 
* **(b) Single feature map:** 빠른 연산이 장점이나, 축소된 정보에서 feature를 꺼낼때 많은 정보 손실이 일어납니다. 즉 예측률이 낮아집니다. 
* **(c) Pyramidal feature hierarchy:** ConvNet에서 처리된 각각의 features들을 사용하는 방법입니다. 문제는 ConvNet의 특징이 처음 이미지에서 feature를 뽑아낼때 점, 선들이다가 딥하게 내려갈수록 얼굴, 자동차같은 형태의 feature들이 뽑히게 됩니다. 즉 그림에서 아래쪽 features들일수록 실제 의미있는 feature들이 없다는게 문제 입니다.
* **(d) Feature Pyramid Network (FPN):** b와 c처럼 빠르지만 더 정확합니다. upsampling을 사용하여 마지막 단계의 features들을 살리면서, 처음단계의 features들까지 사용하게 됩니다.

(d) Feature Pyramid Network 의 경우 top-down pathway를 통해서 semantic 정보를 high resolution으로 reconstruction하게 됩니다. 문제는 downsampling  그리고 upsampling에서 나오는 정확한 위치 정보에 대한 손실이 일어나게 되는데 보완해 주는것이 일치하는 feature maps과 reconstructed layers를 skip connection처럼 연결해주는 것입니다. 

## Semantic Value VS Resolution

Resolution과 semantic value와 반비례하는 관계라고 보면 됩니다. <br>
Bottom layers는 high resolution이지만 semantic value가 부족하여 객체 인식에 적합하지 못하고, <br>
upper layers의 경우는 semantic value가 높지만, low resolution이기 때문에 작은 객체를 못잡아 냅니다.

![Data Flow](images/fpn-semantic.jpeg)

## Image Pyramid VS Convolution Neural Network 

Image pyramid의 경우 이미지마다 다양한 크기의 features들을 뽑아낼수 있지만, 앞서 말했듯이 연산비용이 너무 높습니다. <br>
Deep ConvNet의 경우 feature hierarchy를 레이어마다 연산을 하고, sub-sampling layer마다 다른 크기의 features를 뽑아내기 때문에 이미 multi-scale부분이 내재화 되어 있다고 볼 수 있습니다. 다양한 크기의 feature maps을 뽑아낼수 있지만 있지만 문제는, high-resolution maps일수록 low-level features (점, 선같은것들)를 갖고 있으며 실질적으로 object recognition에 쓰이기에는 문제가 있습니다.

## Single Shot Detector (SSD)

SSD의 ConvNet을 features를 마치 featurized image pyramid처럼 사용하는 초기모델 중의 하나 입니다. <br>
하지만 low-level features을 피하기 위해서 SSD는 upper layers들만 사용합니다. <br>
때문에 여전히 작은 객체는 잡지를 못하고, 연산량은 많습니다.


![SSD](images/fpn-ssd.png)



## Bottom-up Pathway

Buttom-up pathway는 기존 ConvNet에서 생성되는 feature maps이라고 생각하면 됩니다. <br>
stride 값이 2이기 때문에 위로 올라갈때마다 spatial 정보를 1/2 잃게 됩니다.

![Bottom Up](images/fpn-buttom-up.png)

## Top-down Pathway

Top-down pathway은 higher pyramid levels에서 upsampling을 함으로서 spatial information 부족하지만 semantic information은 강한 features 들을 사용함으로서 high resolution을 대략적으로 구현하게 됩니다.

C5 channel depth를 256-d 으로 줄이기 위해서 1 x 1 convolution을 적용하여 M5 가 나오게 됩니다.<br>
이후 3 x 3 convolution을 적용하여 P5를 생성하며, P5는 객체인식을 위한 첫번째 feature map이 되게 됩니다.

Top-down path를 내려가면서, 2만큼 nearest neighbors upsampling을 사용하여 upsampling을 하게 됩니다.<br>
다시 1 x 1 convolution을 상응하는 feature maps (C4)에 적용해준뒤 element-wise로 서로 더해주게 됩니다. <br>
그리고 3 x 3 convolution을 적용하여 두번째 feature maps을 만들게 됩니다. 

이런식으로 반복적으로 최종 feature maps을 생성하면 aliasing effect of upsampling효과를 차단할 수 있습니다.

## FPN with RPN

Head라는 표현으로 기존 RPN을 정의를 하게 되는데, Head는 3 x 3 convolution 그리고 뒤이어서 나오는 1 x 1 convolutions 2개를 (object/non-object binary classification layer 그리고 bounding box regression layer)의미합니다. 

포인트는 각각의 FPN에서 뽑힌 features들마다 HEAD를 적용한다는 것입니다. 



FPN자체는 object detector가 아니고, feature detector (또는 feature extractor)로서 object detector인 RPN과 연결이 됩니다. <br>
FPN에서 추출된 feature maps을 RPN의 3 x 3 convolution layer에 태운후 -> 다시 1 x 1 convolution을 통해서 object detecting class 그리고 bounding box regression을 하게 됩니다.

![FPN RPN](images/fpn-rpn.png)

# Anchors

[Feature Pyramid](https://arxiv.org/abs/1612.03144) 페이퍼에서는 feature pyramid에서 추출하는 각각의 feature maps마다 서로 다른 크기를 갖고 있으므로, single scale을 사용하라고 합니다. <br>
그리고 [RetinaNet](https://arxiv.org/pdf/1708.02002.pdf) 페이퍼에서는 더 조밀하게 객체를 찾기 위해서 $ \{ 2^0, 2^{1/3}, 2^{2/3} \} $  을 적용하라고 합니다. <br>
즉 feature pyramid에서 다양한 크기의 feature maps을 이용해서 같은 객체를 서로 다른 크기로 찾아볼수 있지만, <br>
여기서 한단계 더 나아가 더 자세하게 찾기 위해서 scales까지 적용한다는 뜻입니다.

Anchor의 크기는 **[32, 64, 128, 256, 512]** 를 P3, P4, P5, P6, P7 에 각각 사용해줍니다. <br>
각각의 pyramid level마다 아래의 ratios를 anchors에 적용합니다. <br>

* FPN outputs to use :  $ \{ P_3, P_4, P_5, P_6, P_7 \} $
* anchor areas : $ \{ 32^2, 64^2, 128^2, 256^2, 512^2 \} $ 
* ratios : $ \{ 1:2,\ 1:1,\ 2:1 \}  = \{0.5, 1, 2 \} $
* scales : $ \{ 2^0, 2^{1/3}, 2^{2/3} \} = \{ 1, 1.259, 1.5874 \} $
* 각 레벨마다 Anchors의 갯수 : A = 9 anchors

# Sub Networks

![RetinaNet SubNet](images/retinanet-subnet.png)

## Classification Sub Network

Classfication subnet은 각각의 A anchors 그리고 K object classes에 대한 공간적 위치에 대해서 객체가 존재하는지에 대한 확률을 예측합니다.<br>
해당 subnet은 매우작은 FCN(fully convolutional network)으로서 각 FPN level마다 붙게 됩니다. 

1. 각각의 feature maps with C channels 을 inputs으로 사용
2. 4개의 3 x 3 convolution with C channels 을 연산 -> 각 4개의 convolution 마다 ReLU activation 사용
3. 3 x 3 convolution with KA channels 을 연산 
4. 마지막으로 각각의 위치에 대한 KA binary prediction에 sigmoid function 적용

> 페이퍼에서는 C = 256 그리고 A = 9 을 사용


## Regression Sub Network

Classification subnet과 마찬가지로 FCN (fully convolutional network)를 사용해서 각각의 pyramid level에 적용을 합니다.<br>
구현은 classification subnet과 동일하지만 마지막 레이어가 4A linear output을 내놓는다는 점이 다릅니다. 

각각의 A anchors마다 4 outputs은 ground-truth anchors와 해당 anchors사이의 relative offset을 regress합니다.

## Regression Target

각각의 anchor마다 K one-hot vector of classification targets (K는 객체 클래스의 갯수) 그리고 4-vector of box regression targets 이 주어지게 됩니다.<br>
IoU 값이 0.5 이상이면 객체로 분류를 하고, 그 이하이면 background로 분류를 합니다. (background로 분류되는 anchors는 그냥 무시하면 됨)

Regression값은 Faster-RCNN과는 다르게 단순히 ground-truth anchors와 anchors 사이의 offset값을 구하면 됩니다. 

$$ \begin{align} 
dx_1 &= (x^*_1 - x_1)/w_a  \\
dy_1 &= (y^*_1 - y_1)/h_a  \\
dx_2 &= (x^*_2 - x_2)/w_a  \\
dy_2 &= (y^*_2 - y_2)/h_a  \\
\end{align} $$

> 사실 페이퍼에서는 ground-truth anchors 그리고 anchors 사이의 offset값을 사용하라고 하는데.. <br>
> offset 둘 사이의 차이란 뜻이지만, 구체적으로 어떤 공식을 사용하라고 나와있지는 않습니다. <br> 
> Faster R-CNN에서 tx, ty, tw, th 를 구할때 width 그리고 height 값으로 나눠줬는데.. 이 부분을 차용하였습니다.

# Training RetinaNet

## Model Initialization

RetinaNet에서 initialization은 매우 중요합니다. <br>
기본적으로 binary classification 모델(RetinaNet은 binary cross entropy를 사용)의 경우 $ y = -1 \text{ or } 1 $ 이 동일하게 나오도록 초기화를 해줍니다. 하지만 class imbalance가 있는 경우 frequent class (여기서는 배경)가 전체 loss값을 대부분 차지해버리기 때문에 초기 학습은 instability를 일으키게 됩니다. 

학습이 안정적으로 되게 만들기 위해서 다음과 같이 초기화를 합니다. 특히 classfication subnet의 마지막 레이어가 매우 중요합니다. 

1. ResNet은 ImageNet으로 pre-trained된 모델을 그대로 사용합니다.
2. FPN의 경우 Feature pyramid network 논문에서 말한대로 초기화를 합니다.
3. Subnets의 마지막 레이어를 제외하고 bias = 0 으로 gaussian weight ( $ \sigma = 0.01 $  $ \mu = 0 $ ) 을 사용합니다.
4. Classfication Subnet의 마지막 레이어는 다음과 같이 bias를 설정합니다.

$$ b = -\log( \frac{1-\pi}{\pi})  $$


즉 초기 학습을 할때 모든 anchors들을 ( $ \pi $ 만큼의 신뢰구간 )  foreground에 가깝게 초기화를 시키는 것입니다. <br>
0.01이라는 값이 나오게 된 이유는 대략 foreground objects 대 background objects를 비교했을때 그 비율이 0.01 을 기준으로 하는 것이며 ( $ \frac{\text{foreground}}{\text{background}} = 0.01 $ ), bias값 자체가 foreground에 가깝게 초기화 함으로서, 초기 학습시 학습이 잘 안되는 현상을 막습니다. 


# TODO

https://medium.com/@14prakash/the-intuition-behind-retinanet-eb636755607d